In [21]:
import pandas as pd
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


# 코드 다시 돌리기 위한 seed 고정
import random
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [22]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


# Data Preprocessing

In [23]:
class myModel(nn.Module):
   

    def __init__(self):
        
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

In [24]:
model = MyModel(3, 100).to(device)
model

MyModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)

In [25]:

num_workers = 0
# batch 사이즈 
batch_size = 20
# validation을 따로 정해주기 
valid_size = 0.2

# 데이터를 먼저 normalization 정규화 시킨다. 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# 여기서는 validation을 따로 뽑아준다. 
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# 여기서 다시 샘플링 
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# 완성된 3가지의 loader 
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# Class 분류하기 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [14]:
#train_transform = transforms.Compose([    
  #  transforms.ToTensor(),
  #  transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
#])        

#test_transform = transforms.Compose([
    #transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
#])    

#train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=train_transform)
#test = torchvision.datasets.CIFAR100(root="./", train=False, download=True, transform=test_transform)

#train_loader = torch.utils.data.DataLoader(train, batch_size=256,
                                          # shuffle=True, num_workers=2)
#test_loader = torch.utils.data.DataLoader(test, batch_size=256,
                                         # shuffle=False, num_workers=2)

#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
#criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [26]:
import torch.optim as optim
# specify loss function
criterion = nn.CrossEntropyLoss()
# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=.01)

In [27]:
# train set에 대하여 
# number of epochs to train the model
n_epochs = [*range(30)]
#List to store loss to visualize
train_losslist = []
valid_loss_min = np.Inf # track change in validation loss

for epoch in range(100):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # tensor를 gpu 로 가능하다면!
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        # batch에 대한 Loss 계산하기 
        loss = criterion(output, target)
        # parameter에대한 loss backward
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
       
        loss = criterion(output, target)
        # val loss 계산
        valid_loss += loss.item()*data.size(0)
    
    # 평균적인 Loss 계산
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    train_losslist.append(train_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss
plt.plot(n_epochs, train_losslist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Performance of Model")
plt.show()

Epoch: 0 	Training Loss: 1.834469 	Validation Loss: 0.373143
Validation loss decreased (inf --> 0.373143).  Saving model ...
Epoch: 1 	Training Loss: 1.365755 	Validation Loss: 0.317855
Validation loss decreased (0.373143 --> 0.317855).  Saving model ...
Epoch: 2 	Training Loss: 1.227948 	Validation Loss: 0.292450
Validation loss decreased (0.317855 --> 0.292450).  Saving model ...
Epoch: 3 	Training Loss: 1.139733 	Validation Loss: 0.284260
Validation loss decreased (0.292450 --> 0.284260).  Saving model ...
Epoch: 4 	Training Loss: 1.072112 	Validation Loss: 0.264994
Validation loss decreased (0.284260 --> 0.264994).  Saving model ...
Epoch: 5 	Training Loss: 1.014079 	Validation Loss: 0.264806
Validation loss decreased (0.264994 --> 0.264806).  Saving model ...
Epoch: 6 	Training Loss: 0.965680 	Validation Loss: 0.242882
Validation loss decreased (0.264806 --> 0.242882).  Saving model ...
Epoch: 7 	Training Loss: 0.924235 	Validation Loss: 0.242166
Validation loss decreased (0.24288

NameError: name 'plt' is not defined

In [28]:
model.load_state_dict(torch.load('model_cifar.pt'))
# 모델저장 

<All keys matched successfully>

In [29]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('test performance' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.060899

Test Accuracy of airplane: 62% (624/1000)
Test Accuracy of automobile: 77% (777/1000)
Test Accuracy of  bird: 54% (545/1000)
Test Accuracy of   cat: 42% (424/1000)
Test Accuracy of  deer: 53% (534/1000)
Test Accuracy of   dog: 47% (471/1000)
Test Accuracy of  frog: 70% (709/1000)
Test Accuracy of horse: 73% (736/1000)
Test Accuracy of  ship: 75% (759/1000)
Test Accuracy of truck: 74% (745/1000)

Test Accuracy (Overall): 63% (6324/10000)
